## Problem Set 2 - Map-Reduce II

In [ ]:
from mooclib import *
import inspect, urllib
html, auth_code, userinfo = google_authenticate(PORT_NUMBER=8111)
html

## <font style="color: red"> Esta práctica contiene 3 ejercicios <!-- ONLY IN NOTEBOOK -->

### Ejercicio 1: Índice invertido

Completa la clase `InvertedIndex` implementando un programa map-reduce que cree un indice invertido para una lista de documentos. Es decir, para cada palabra en nuestra colección de documentos el índice invertido no da en qué documentos aparece. Estos índices se usan posteriormente para responder a queries:


_Ejemplo de ejecución_ (entrada es en formato **json**)

    ["001", "the car is nice"]
    ["002", "that car is mine"]
    ["003", "that shirt is nice"]  
    ["004", "the car is the best"]
    
_Resultado_

    "best"  ["004"]
    "car"   ["001", "002", "004"]
    "is"    ["001", "002", "003", "004"]
    "mine"  ["002"]
    "nice"  ["001", "003"]
    "shirt" ["003"]
    "that"  ["002", "003"]
    "the"   ["001", "004"]

_Nota: usa la linea:_

    key, text = json.loads(line)

_para obtener extraer el id del documento y el texto del mismo del formato **json** _ 

In [ ]:
%%writefile files/p_02B_01_InvertedIndex.py
from mrjob.job import MRJob
import json

class InvertedIndex(MRJob):

    def mapper(self, _, line):
        key, text = json.loads(line)
        # TU CODIGO AQUI
        yield "KEY", "VALUE"

    def reducer(self, key, values):
        docs = []
        # TU CODIGO AQUI
        yield "KEY", "VALUE"
        
if __name__ == '__main__':
    InvertedIndex.run()

verifica que la salida es la esperada y prueba con nuevos valores. <font style="color: red">_Elimina la opción_ `--quiet` _para ver los mensajes de error de tu código si no produce la salida esperada_


In [ ]:
%%script /opt/anaconda/bin/python files/p_02B_01_InvertedIndex.py -q 
["001", "the car is nice"]
["002", "that car is mine"]
["003", "that shirt is nice"]  
["004", "the car is the best"]

In [ ]:
%%script /opt/anaconda/bin/python files/p_02B_01_InvertedIndex.py -q 
["001", "War is a state of armed conflict between autonomous organizations (such as states and non-state actors) or coalitions of such organizations. It is generally characterized by extreme collective aggression, destruction, and usually high mortality. The set of techniques used by a group to carry out war is known as warfare. An absence of war is usually called peace."]
["032", "War must entail some degree of confrontation using weapons and other military technology and equipment by armed forces employing military tactics and operational art within a broad military strategy subject to military logistics."]
["105", "While some scholars see warfare as a universal and ancestral aspect of human nature, others argue that it is only a result of specific socio-cultural or ecological circumstances"]

### Ejercicio 2: Amigos asimétricos

Sobre el mismo ejemplo de las notas queremos ahora obtener una lista de parejas en las que la relación de amistad es asimétrica. Es decir, si en la entrada tenemos (personA, personB) y también tenemos (personB, personA) entonces ninguna de esas dos tuplas ha de aparecer en la salida. En cambio, si en la entrada tenemos (personA, personB) pero NO TENEMOS (personB, personA), entonces en la salida han de aparecer AMBAS TUPLAS.

_Ejemplo de ejecución_ (entrada es en formato **json**)

    ["juan", "pepe"]
    ["juan", "sebastian"]
    ["raul", "pepe"]
    ["juan", "ana"]
    ["ana", "pepe"]
    ["pepe", "ana"]
    ["ana", "juan"]
    ["ana", "pedro"]

_Resultado_

    "ana"   "pedro"
    "juan"  "pepe"
    "juan"  "sebastian"
    "pedro" "ana" 
    "pepe"  "juan"
    "pepe"  "raul"
    "raul"  "pepe"
    "sebastian" "juan"



In [2]:
%%writefile files/p_02B_02_FriendsAsimetric.py
from mrjob.job import MRJob
from mrjob.step import MRStep

import json

class FriendsAsimetric(MRJob):

    def mapper(self, _, line):
        f1, f2 = json.loads(line)
        # TU CODIGO AQUI
        yield "KEY", "VALUE"

    def reducer(self, key, values):
        # TU CODIGO AQUI
        yield "KEY", "VALUE"
        
if __name__ == '__main__':
    FriendsAsimetric.run()

Overwriting files/p_02B_02_FriendsAsimetric.py


In [3]:
%%script /opt/anaconda/bin/python files/p_02B_02_FriendsAsimetric.py -q
["juan", "pepe"]
["juan", "sebastian"]
["raul", "pepe"]
["juan", "ana"]
["ana", "pepe"]
["pepe", "ana"]
["ana", "juan"]
["ana", "pedro"]

"KEY"	"VALUE"


### Ejercicio 3: Matriz dispersa

Representamos dos matrices dispersas **a** y **b** de 2x2 de la siguiente forma:

    ["a", 0, 0, 63]
    ["a", 1, 1, 32]
    ["b", 0, 0, 69]
    ["b", 0, 1, 18]
    ["b", 1, 1, 28]

es decir

    a = [63  0]     b = [69 18]    a x b = [4347 1134]
        [ 0 32]         [ 0 28]            [   0  896]
        
        
tienes que implementar un programa map-reduce que acepte como entrada las matrices en el formato disperso y devuelva la multiplicación de ambas matrices en formato también disperso.

_Ejemplo de ejecución_ (entrada es en formato **json**)

    ["a", 0, 0, 63]
    ["a", 1, 1, 32]
    ["b", 0, 0, 69]
    ["b", 0, 1, 18]
    ["b", 1, 1, 28]

_Resultado_

    [0, 0]	4347
    [0, 1]	1134
    [1, 1]	896

_Ayuda: Seudocódigo del map, entradas reduce_

    map:
       foreach position (i,j) in A for k=1..N yield ( (i,k), A[i,j]) 
       foreach position (j,k) in B for i=1..L yield ( (i,j), B[j,k]) 
 
    entrada reduce
    key      values
    [0, 0]   [[0, 63], [0, 69]]
    [0, 1]   [[0, 18], [0, 63], [1, 28]]
    [1, 0]   [[0, 69], [1, 32]]
    [1, 1]   [[0, 18], [1, 28], [1, 32]]    
 

In [ ]:
%%writefile files/p_02B_03_Matmult.py
from mrjob.job import MRJob
import json

N=2 # dimension de las matrices cuadradas a multiplicar

class Matmult(MRJob):

    def mapper(self, _, line):
        id, y, x, val = json.loads(line)
        # TU CODIGO AQUI
        yield "KEY", "VALUE"
        
    def reducer(self, key, values):
        # TU CODIGO AQUI
        yield "KEY", "VALUE"
        
if __name__ == '__main__':
    Matmult.run()

In [ ]:
%%script /opt/anaconda/bin/python files/p_02B_03_Matmult.py -q
["a", 0, 0, 63]
["a", 1, 1, 32]
["b", 0, 0, 69]
["b", 0, 1, 18]
["b", 1, 1, 28]

---

**Obtén la calificación de esta práctica**

In [ ]:
%run -i code/p_02B_eval